[Podatkovno zbirko](https://www.clarin.si/repository/xmlui/handle/11356/1054) z označenimi sentimenti objav s Twitterja smo pridobili s portala Clarin.si V prvem delu se lotimo pridobitve besedila vseh objav v podatkovni zbirki, saj so v prvotni obliki twitti identificirani le z ID-jem in ne s celotnim besedilom objave. V drugem delu se lotimo preprocesiranja podatkov.

## Pridobivanje podatkov s pomočjo Twitter API-ja Tweepy
Na portalu Twitter developer smo si ustvarili račun in pridobili ključe za uporabo Tweepy API-ja.
### Tweepy API ključi

In [1]:
#!pip install tweepy
import tweepy

In [2]:
consumer_key = 'tJJWBrVL5ePKzrNMd0upUNCkx'
consumer_secret = 'wpb4Jx04ZufeZCIAdpLqSZfpIFO01OvU6F2jwM5QxoL8uDPKC2'
access_token = '1549038171714998272-4176ZxEc2amIJWhBJMM8NtstEzWBoC'
access_token_secret = 'GQP0HFsWTYN8JEbr9yK47ZJMCzGBHkUsyuatZLsszoPuA'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

Naložimo datoteki, pridobljeni s portala Clarin: eno v slovenskem in drugo v angleškem jeziku. Prvo bomo uporabili kot zbirko za učenje modela (angl. *training set*), drugo pa kot zbirko za testiranje (angl. *test set*). Vsako datoteko posebej shranimo v seznam, pri čemer je vsak posamezni twitt sestavljen iz slovarja *{'ID': id_twitta, 'sent': sentiment_twitta}*.

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving English_Twitter_sentiment.csv to English_Twitter_sentiment.csv
Saving Slovenian_Twitter_sentiment.csv to Slovenian_Twitter_sentiment.csv
User uploaded file "English_Twitter_sentiment.csv" with length 3353147 bytes
User uploaded file "Slovenian_Twitter_sentiment.csv" with length 4356752 bytes


### Shranjevanje korpusov v slovenščini in angleščini v seznam

In [4]:
import csv
corpusSlo = []

with open('Slovenian_Twitter_sentiment.csv', 'r') as f:
    lineReader = csv.reader(f, delimiter=',', quotechar="\"")
    for row in lineReader:
        if row:
            #print(row)
            corpusSlo.append({'ID': row[0], 'sent': row[1]})

In [5]:
corpusSlo[0]

{'ID': 'TweetID', 'sent': 'HandLabel'}

In [6]:
corpusSlo = corpusSlo[1:]

In [7]:
len(corpusSlo)

133935

In [8]:
corpusEng = []

with open('English_Twitter_sentiment.csv', 'r') as f:
  lineReader = csv.reader(f, delimiter=',', quotechar="\"")
  for row in lineReader:
    if row:
      corpusEng.append({'ID': row[0], 'sent': row[1]})

In [9]:
corpusEng = corpusEng[1:]

In [10]:
len(corpusEng)

103034

### Pridobivanje besedila objav s Twitterja za slovensko podatkovno zbirko
Zapise, ki jih pridobimo s pomočjo Tweepy-ja, sproti zapisujemo v novo ustvarjeno tekstovno datoteko. Ta del nam je vzel dva dneva, saj ima vsak Twitter developer limit twittov, ki jih lahko pridobi v določenem času. Na vsakih 8000 pridobljenih twittov je funkcija mirovala približno 15 minut. Pridobljene twitte smo shranili v tekstovno datoteko raje kot takoj v seznam, da bi se izognili primeru, če bi seja delovnega zvezka iz kakršnekoli napake potekla in bi izgubli že do sedaj pridobljene objave.

In [ ]:
t = ''
i = 0

with open('tweetsSlo.txt', 'w') as f:
    for tweet in corpusSlo[:60000]:
        id = tweet['ID']
        #print(id)
        try:
            fetched = api.get_status(id)
            t = fetched.text

            if "\n" in t:
              t = t.replace("\n", " ")

            #print(i, t, id)

        except tweepy.TweepError as e: 
            t = e
            #print(i, t, id)

        i += 1
        f.write(i, t, id)

In [55]:
tweetsSlo = []

In [13]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving tweetsSlo.txt to tweetsSlo.txt
User uploaded file "tweetsSlo.txt" with length 6835021 bytes


Iteriramo čez tekstovno datoteko s pridobljenimi twitti ter jih shranimo v seznam. Dolžina seznama je 58700 twittov. Podatke nato shranimo v tabelo s pomočjo knjižnice *Pandas*.

In [56]:
import re
import csv

with open('tweetsSlo.txt', 'r') as f:
  lineReader = csv.reader(f, delimiter=',', quotechar="\"")
  for row in lineReader:
    if row:
      row = ' '.join(row)
      tweetsSlo.append(row)

In [57]:
len(tweetsSlo)

58700

In [58]:
import pandas as pd
df = pd.DataFrame(tweetsSlo, index=None, columns=['Tweet'])

In [59]:
df.shape

(58700, 1)

### Preprocesiranje in čiščenje besedila zbirke v slovenskem jeziku
Poiščemo indekse vseh vrstic, ki namesto besedil objav s Twitterja vsebujejo besedila napak: doseženih limitov pridobljenih twittov ali napak, ki povedo, da račun objavljenega twitta ne obstaja več. Besedilo pretvorimo v majhne črke ter odstranimo morebitne dodatne presledke. Odstranimo vrstice, ki vsebujejo napake o doseženem limitu.

In [60]:
str = "Rate limit reached"
limiti = []
errorji = []

for i in range(len(df['Tweet'])):
  #print(i)
  elementi = df['Tweet'][i].split()
  if str in df['Tweet'][i]:
    limiti.append(i)
    df['Tweet'][i-1] += " " + df['Tweet'][i]
  elif elementi[1] == "[{'code':":
    errorji.append(i)

In [61]:
df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join(x.lower().split()))

In [62]:
df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join(x.split()))

In [63]:
df.head(30)

,Tweet
0,0 [{'code': 144 'message': 'no status found wi...
1,1 [{'code': 144 'message': 'no status found wi...
2,2 [{'code': 144 'message': 'no status found wi...
3,"3 .@matevz_zupancic: ""za uniona olimpijo vedno..."
4,4 [{'code': 63 'message': 'user has been suspe...
5,5 rt @revijareporter: znp: preganja se novinar...
6,6 ste zamudili spletno razpravo z @vivianeredi...
7,7 rt @knjiznicanuk: #nadanasnjidan: 1909 je za...
8,8 @ajudez ima samo ni čist to kar hočem... 424...
9,9 @ajudez ima samo ni čist to kar hočem... 424...


In [64]:
df.drop(limiti,0,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


Tabeli dodamo stolpec s z oznakami sentimenta za posamezno objavo.

In [65]:
sentimentSlo = []
for tweet in corpusSlo:
  sent = tweet['sent']
  sentimentSlo.append(sent)

In [66]:
sentimentSlo = sentimentSlo[:df.shape[0]]

In [67]:
df['Sentiment'] = sentimentSlo

In [68]:
len(df['Tweet'])

58645

In [69]:
len(df['Sentiment'])

58645

Iz posamezne vrstice v besedilu twitta odstranimo število na začetku. Nato odstranimo tudi ID na koncu posamezne vrstice ter izbrišemo vse vrstice, ki vsebujejo besedila o doseženih napakah: uporabnik ne obstaja, uporabnik je bil blokiran, objava ne obstaja.

In [70]:
tweets = []

for tweet in df['Tweet']:
  elementi = tweet.split()
  #print(elementi)
  if elementi[0].isdigit():
    elementi = elementi[1:]
    tweets.append(' '.join(elementi))
  else:
    tweets.append(tweet)

In [71]:
df['Tweet'] = tweets

In [72]:
tweets = []

for tweet in df['Tweet']:
  elementi = tweet.split()
  elementi = elementi[:-1]
  tweets.append(' '.join(elementi))


In [73]:
df['Tweet'] = tweets

In [74]:
df.head(30)

,Tweet,Sentiment
0,[{'code': 144 'message': 'no status found with...,Neutral
1,[{'code': 144 'message': 'no status found with...,Neutral
2,[{'code': 144 'message': 'no status found with...,Positive
3,".@matevz_zupancic: ""za uniona olimpijo vedno c...",Positive
4,[{'code': 63 'message': 'user has been suspend...,Negative
5,rt @revijareporter: znp: preganja se novinarje...,Negative
6,ste zamudili spletno razpravo z @vivianereding...,Neutral
7,rt @knjiznicanuk: #nadanasnjidan: 1909 je zapa...,Neutral
8,@ajudez ima samo ni čist to kar hočem...,Neutral
9,@ajudez ima samo ni čist to kar hočem...,Negative


In [75]:
df.drop(errorji,0,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [76]:
df.shape

(42748, 2)

Iz besedil twittov odstranimo vse *@označbeProfilov*, *#ključnike*, elektronsko pošto, URL naslove, *retweete* ter vse posebne simbole.

In [77]:
#remove mentions
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'#([a-zA-Z0-9_]{1,50})', '', x).strip())

In [78]:
#remove hashtags
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'@([a-zA-Z0-9_]{1,50})', '', x).strip())

In [79]:
#remove emails
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)', "", x))

In [80]:
#remove URLs
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', "", x))

In [81]:
#remove RTs
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'\brt\b', '', x).strip())

In [82]:
#remove special characters and punctuation
df['Tweet'] = df['Tweet'].apply(lambda x: re.sub(r'[^\w ]+', "", x))

In [83]:
#remove multiple spaces
df['Tweet'] = df['Tweet'].apply(lambda x: ' '.join(x.split()))

In [84]:
df = df.reset_index(drop=True)

Preverimo, če obstajajo vrstice, ki imajo vrednost v stolpcu *Tweet* ali *Sentiment* prazno vrednost in take vrstice odstranimo.

In [85]:
prazniS = []
for i in range(len(df['Sentiment'])):
  s = df['Sentiment'][i]
  if not s:
    prazniS.append(i)

In [86]:
df.drop(prazniS,0,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [87]:
df = df.reset_index(drop=True)

In [88]:
prazniText = []
for i in range(len(df['Tweet'])):
  s = df['Tweet'][i]
  if not s:
    prazniText.append(i)

In [89]:
prazniText

[15427, 15870, 22878, 22879, 26392, 32789, 34379, 36546, 38535, 40253, 41226]

In [90]:
df.drop(prazniText,0, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [91]:
df = df.reset_index(drop=True)

In [92]:
df.head(30)

,Tweet,Sentiment
0,za uniona olimpijo vedno cilj le en naslov prv...,Positive
1,znp preganja se novinarje ne pa tudi tistih ki...,Negative
2,ste zamudili spletno razpravo z oglejte si pos...,Neutral
3,1909 je zapadel v radovljici visok sneg domači...,Neutral
4,ima samo ni čist to kar hočem,Neutral
5,ima samo ni čist to kar hočem,Negative
6,lol evo skrbimo da ti tudi v mrkih trenutkih o...,Positive
7,ubistvu mi je bil synology na prvem mestu sam ...,Positive
8,1909 je zapadel v radovljici visok sneg domači...,Neutral
9,čez dve uri od 15 do 17 ure 1712014 in prihran...,Neutral


Pridobljeno prečiščeno tabelo shranimo v novo csv datoteko.

In [93]:
df.to_csv('sentTweetsSlo.csv', encoding = 'utf-8-sig', index=False) 
files.download('sentTweetsSlo.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Pridobivanje besedila objav s Twitterja za slovensko podatkovno zbirko
Po enakem postopku se lotimo pridobivanja objav v angleškem jeziku. Ker bomo to zbirko uporabili za testiranje, ki bo z zbirko za učenje v razmerju 2:8, pridobimo manj objav kot za slovensko, zato nam je ta del vzel le en dan.

In [ ]:
t = ''
i = 0

with open('tweetsEng.txt', 'w'):
    for tweet in corpusEng[:35000]:
        id = tweet['ID']
        #print(id)
        try:
            fetched = api.get_status(id)
            t = fetched.text

            if "\n" in t:
              t = t.replace("\n", " ")

            #print(i, t, id)

        except tweepy.TweepError as e: 
            t = e
            #print(i, t, id)

        i += 1
        #tweetsSlo.append({'id': id, 'text': t})
        f.write(i, t, id)

In [94]:
tweetsEng = []

In [95]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving tweetsEng.txt to tweetsEng.txt
User uploaded file "tweetsEng.txt" with length 3450227 bytes


In [96]:
import re

with open('tweetsEng.txt', 'r') as f:
  lineReader = csv.reader(f, delimiter=',', quotechar="\"")
  for row in lineReader:
    if row:
      row = ' '.join(row)
      tweetsEng.append(row)

In [97]:
len(tweetsEng)

34812

In [98]:
dfEng = pd.DataFrame(tweetsEng, index=None, columns=['Tweet'])

In [99]:
dfEng.shape

(34812, 1)

### Preprocesiranje in čiščenje besedila zbirke v angleškem jeziku
Postopek je enak tistemu, ki smo za uporabilo za procesiranje podatkovne zbirke v slovenščini.

In [100]:
str = "Rate limit reached"
limitiEng = []
errorjiEng = []

for i in range(len(dfEng['Tweet'])):
  #print(i)
  elementi = dfEng['Tweet'][i].split()
  if str in dfEng['Tweet'][i]:
    limitiEng.append(i)
    dfEng['Tweet'][i-1] += " " + dfEng['Tweet'][i]
  elif elementi[1] == "[{'code':":
    errorjiEng.append(i)

In [101]:
dfEng['Tweet'] = dfEng['Tweet'].apply(lambda x: ' '.join(x.lower().split()))

In [102]:
dfEng['Tweet'] = dfEng['Tweet'].apply(lambda x: ' '.join(x.split()))

In [103]:
dfEng.drop(limitiEng,0,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [104]:
dfEng.shape

(34790, 1)

In [105]:
sentimentEng = []
for tweet in corpusEng:
  sent = tweet['sent']
  sentimentEng.append(sent)

In [106]:
sentimentEng = sentimentEng[:dfEng.shape[0]]

In [107]:
dfEng['Sentiment'] = sentimentEng

In [108]:
len(dfEng['Tweet'])

34790

In [109]:
len(dfEng['Sentiment'])

34790

In [110]:
tweetsEng = []

for tweet in dfEng['Tweet']:
  elementi = tweet.split()
  #print(elementi)
  if elementi[0].isdigit():
    elementi = elementi[1:]
    tweetsEng.append(' '.join(elementi))
  else:
    tweetsEng.append(tweet)

In [111]:
dfEng['Tweet'] = tweetsEng

In [112]:
tweetsEng = []

for tweet in dfEng['Tweet']:
  elementi = tweet.split()
  elementi = elementi[:-1]
  tweetsEng.append(' '.join(elementi))

In [113]:
dfEng['Tweet'] = tweetsEng

In [114]:
dfEng.head(30)

,Tweet,Sentiment
0,[{'code': 144 'message': 'no status found with...,Negative
1,[{'code': 144 'message': 'no status found with...,Positive
2,[{'code': 179 'message': 'sorry you are not au...,Positive
3,rt @cjayytaughther: rule #1: never be #2,Neutral
4,[{'code': 179 'message': 'sorry you are not au...,Neutral
5,[{'code': 144 'message': 'no status found with...,Positive
6,[{'code': 63 'message': 'user has been suspend...,Neutral
7,puff puff pass ya hoe,Negative
8,0808 mon lit,Neutral
9,@jthizzle when was this,Neutral


In [115]:
dfEng.drop(errorjiEng,0,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [116]:
dfEng.shape

(16715, 2)

In [117]:
#remove mentions
dfEng['Tweet'] = dfEng['Tweet'].apply(lambda x: re.sub(r'#([a-zA-Z0-9_]{1,50})', '', x).strip())

In [118]:
#remove hashtags
dfEng['Tweet'] = dfEng['Tweet'].apply(lambda x: re.sub(r'@([a-zA-Z0-9_]{1,50})', '', x).strip())

In [119]:
#remove emails
dfEng['Tweet'] = dfEng['Tweet'].apply(lambda x: re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)', "", x))

In [120]:
#remove URLs
dfEng['Tweet'] = dfEng['Tweet'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', "", x))

In [121]:
#remove RTs
dfEng['Tweet'] = dfEng['Tweet'].apply(lambda x: re.sub(r'\brt\b', '', x).strip())

In [122]:
#remove special characters and punctuation
dfEng['Tweet'] = dfEng['Tweet'].apply(lambda x: re.sub(r'[^\w ]+', "", x))

In [123]:
#remove multiple spaces
dfEng['Tweet'] = dfEng['Tweet'].apply(lambda x: ' '.join(x.split()))

In [124]:
dfEng = dfEng.reset_index(drop=True)

In [125]:
prazniS = []
for i in range(len(dfEng['Sentiment'])):
  s = dfEng['Sentiment'][i]
  if not s:
    prazniS.append(i)

In [126]:
dfEng.drop(prazniS,0,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [127]:
dfEng = dfEng.reset_index(drop=True)

In [128]:
prazniText = []
for i in range(len(dfEng['Tweet'])):
  s = dfEng['Tweet'][i]
  if not s:
    prazniText.append(i)

In [129]:
prazniText

[649, 3490, 11325]

In [130]:
dfEng.drop(prazniText,0,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [131]:
dfEng = dfEng.reset_index(drop=True)

In [134]:
dfEng.shape

(16712, 2)

In [132]:
dfEng.head(30)

,Tweet,Sentiment
0,rule never be,Neutral
1,puff puff pass ya hoe,Negative
2,0808 mon lit,Neutral
3,when was this,Neutral
4,when was this,Neutral
5,i cant believe mickey mouse had sex with shado...,Neutral
6,tgim thank god its monday baby,Negative
7,peace love shea,Neutral
8,same,Neutral
9,otaku loving someone who doesnt exist kpop fan...,Negative


In [133]:
dfEng.to_csv('sentTweetsEng.csv', encoding = 'utf-8-sig', index=False) 
files.download('sentTweetsEng.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>